In [ ]:
import sklearn
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from snowflake.snowpark.context import get_active_session

In [ ]:
session = get_active_session()
df = session.table('DW_RESEARCH.MLTABLES.TITANIC').to_pandas()

df.head(6) # top 6 records

In [ ]:
numSamples = len(df)

df.keys() # feature names - SURVIVED is a classification, won't work with linear regression without logistic regression. We will attempt to determine FARE
numFeatures = len(df.keys()) - 1 # one of the keys is the result
features = df.keys().drop("FARE").to_list()

print(f"""num samples={numSamples}, num features={numFeatures}""")
print(f"""features={features}""")

x = np.array(df["AGE"])
y = np.array(df["FARE"])
print("The average fare was $%.2f." % np.mean(y))

In [ ]:
print(x)

you will notice that we null ages, we need to remove these. always a good idea to go through the data and do atleast a minimal amount of data cleaning.

In [ ]:
# getting rid of instances where age is nan and there is fare data
mask = ~np.isnan(x)
x = x[mask]
y = y[mask]

In [ ]:
plt.scatter(x,y)
plt.xlabel("Age (years)")
plt.ylabel("Fare ($)")
plt.grid(True)

Notice that there is no distinct visual way to determine whether there is a trend between age and fare. Keep this in mind as we are looking at two possibly unrelated topics.

### How are we determining the "best" regression for our data so far?

Residuals. This is the difference between the actual data and our prediction.\
$$\epsilon_{i}=y_{i}-ŷ_{i}$$

We want to find the model parameters that best minimize the residuals, one option is residual sum of squares (RSS).

#### RSS - Residual Sum of Squares
This is often also called the sum of squared errors (SSE) or sum of square residuals (SSR).\
$$RSS\left(\beta_{0},\beta_{1}\right)=\sum_{i=1}^{n}\epsilon_{i}^{2}=\sum_{i=1}^{n}\left(y_{i}-ŷ_{i}\right)^{2}=\sum_{i=1}^{n}\left(y_{i}-\left(\beta_{1}x_{i}+\beta_{0}\right)\right)^{2}$$\
The values of $$\beta$$ that minimizes RSS is the least-squares fit.

#### Where is this heading?
The general approach to a ML problem?
* A model with some parameters
* Get training data
* Choose a loss function
* Find parameters that minimize that loss (this is where we would use SGD to optimize loss)

In [ ]:
def fit_linear(x,y):
    xm = np.mean(x) # feature mean
    ym = np.mean(y) # target mean

    sxx = np.mean((x-xm)**2) # sample variance
    syy = np.mean((y-ym)**2)
    sxy = np.mean((x-xm)*(y-ym))

    beta1 = sxy/sxx
    beta0 = ym - beta1*xm
    rsq = sxy**2/sxx/syy

    return beta0, beta1, rsq

# How do we derive this?
## Quick Statistics:
### Sample mean:
* $$\overline{x}=\frac{1}{n}\sum_{i=1}^{n}x_{i}$$
* $$\overline{y}=\frac{1}{n}\sum_{i=1}^{n}y_{i}$$

### Sample variance:
* $$s_{x}^{2}=\frac{1}{n}\sum_{i=1}^{n}\left(x_{i}-\overline{x}\right)^{2}$$
* $$s_{y}^{2}=\frac{1}{n}\sum_{i=1}^{n}\left(y_{i}-\overline{y}\right)^{2}$$

***Sample standard deviation can be found by square-rooting the sample variance.***

### Sample covariance:
* $$s_{xy}=\sum_{i=1}^{n}\left(x_{i}-\overline{x}\right)\left(y_{i}-\overline{y}\right)$$
  + *this indicates how "related" $$x_{i}$$ and $$y_{i}$$ are*
---
## Minimizing the RSS:
To minimize the $$RSS\left(\beta_{0},\beta_{1}\right)$$, we find the $$\beta_{0}$$ and $$\beta_{1}$$ that zero the gradient.

What is the gradient? You can think of it as the vector of partial derivatives. A partial derivative, somewhat similar to normal derivatives in calculus, is the rate of change of one variable while holding all other variables constant. 

$$\left[\frac{∂RSS\left(\beta_{0},\beta_{1}\right)}{∂\beta_{0}},\ \frac{∂RSS\left(\beta_{0},\beta_{1}\right)}{∂\beta_{1}}\right]^{⊤}=\begin{bmatrix} 0 \\ 0 \end{bmatrix}$$ 

### Deriving $$\beta_{0}$$:
$$0=\frac{∂RSS\left(\beta_{0},\beta_{1}\right)}{∂\beta_{0}}=\sum_{i}^{ }\frac{∂}{∂\beta_{0}}\left(y_{i}-\beta_{0}-\beta_{1}x_{i}\right)^{2}=\sum_{i}^{ }-2\left(y_{i}-\beta_{0}-\beta_{1}x_{i}\right)$$

$$0=\sum_{i}^{ }-2\left(y_{i}-\beta_{0}-\beta_{1}x_{i}\right)$$ **→** $$0\left(-\frac{1}{2n}\right)=-\frac{1}{2n}\left(\sum_{i}^{ }-2\left(y_{i}-\beta_{0}-\beta_{1}x_{i}\right)\right)$$ 

$$0=-\frac{1}{2n}\left(\sum_{i}^{ }-2\left(y_{i}-\beta_{0}-\beta_{1}x_{i}\right)\right)=\frac{1}{n}\sum_{i}^{n}\left(y_{i}-\beta_{0}-\beta_{1}x_{i}\right)=\frac{1}{n}\sum_{i=1}^{n}y_{i}-\beta_{0}-\beta_{1}\frac{1}{n}\sum_{i=1}^{n}x_{i}=\overline{y}-\beta_{0}-\beta_{1}\overline{x}$$

$$0=\overline{y}-\beta_{0}-\beta_{1}\overline{x}$$ **↔** $$\beta_{0}=\overline{y}-\beta_{1}\overline{x}$$\

### Deriving $$\beta_{1}$$:
$$0=\frac{∂RSS\left(\beta_{0},\beta_{1}\right)}{∂\beta_{0}}=\sum_{i}^{ }\frac{∂}{∂\beta_{1}}\left(y_{i}-\beta_{0}-\beta_{1}x_{i}\right)^{2}=\sum_{i}^{ }-2\left(y_{i}-\beta_{0}-\beta_{1}x_{i}\right)x_{i}$$

$$0=\sum_{i}^{ }-2\left(y_{i}-\beta_{0}-\beta_{1}x_{i}\right)x_{i}$$ **→** $$0\left(-\frac{1}{2n}\right)=-\frac{1}{2n}\left(\sum_{i}^{ }-2\left(y_{i}-\beta_{0}-\beta_{1}x_{i}\right)x_{i}\right)$$

$$0=-\frac{1}{2n}\left(\sum_{i=1}^{n}-2\left(y_{i}-\beta_{0}-\beta_{1}x_{i}\right)x_{i}\right)=\frac{1}{n}\sum_{i=1}^{n}x_{i}\left(y_{i}-\beta_{0}-\beta_{1}x_{i}\right)=\frac{1}{n}\sum_{i=1}^{n}x_{i}y_{i}-\frac{1}{n}\sum_{i=1}^{n}x_{i}\beta_{0}-\beta_{1}\frac{1}{n}\sum_{i=1}^{n}x_{i}^{2}=\frac{1}{n}\sum_{i=1}^{n}x_{i}y_{i}-\overline{x}\beta_{0}-\beta_{1}\frac{1}{n}\sum_{i=1}^{n}x_{i}^{2}$$

Since we have already derived $$\beta_{0}$$ in terms of $$\beta_{1}$$ we can plug in our derivation in for $$\beta_{0}$$ in 

$$0=\frac{1}{n}\sum_{i=1}^{n}x_{i}y_{i}-\overline{x}\beta_{0}-\beta_{1}\frac{1}{n}\sum_{i=1}^{n}x_{i}^{2}$$

$$0=\frac{1}{n}\sum_{i=1}^{n}x_{i}y_{i}-\overline{x}\beta_{0}-\beta_{1}\frac{1}{n}\sum_{i=1}^{n}x_{i}^{2}=\frac{1}{n}\sum_{i=1}^{n}x_{i}y_{i}-\overline{x}\left(\overline{y}-\beta_{1}\overline{x}\right)-\beta_{1}\frac{1}{n}\sum_{i=1}^{n}x_{i}^{2}=\frac{1}{n}\sum_{i=1}^{n}x_{i}y_{i}-\overline{x}\overline{y}-\beta_{1}\left(\frac{1}{n}\sum_{i=1}^{n}x_{i}^{2}-\overline{x}^{2}\right)=s_{xy}-\beta_{1}\left(s_{x}^{2}\right)$$

$$0=s_{xy}-\beta_{1}\left(s_{x}^{2}\right)$$ **↔** $$\beta_{1}=\frac{s_{xy}}{s_{x}^{2}}$$

---
## Finding the minimum RSS
To find the minimum RSS, we can plug the optimal value of $$\beta_{0}$$ and $$\beta_{1}$$ into the RSS formula we had earlier:

$$RSS\left(\beta_{0},\beta_{1}\right)=\sum_{i}^{ }\left(y_{i}-\beta_{0}-\beta_{1}x_{i}\right)^{2}$$

$$RSS\left(\beta_{0},\beta_{1}\right)=\sum_{i}^{ }\left(y_{i}-\beta_{0}-\beta_{1}x_{i}\right)^{2}=\sum_{i}^{ }\left(\left(y_{i}-\overline{y}\right)-\beta_{1}\left(x_{i}-\overline{x}\right)\right)^{2}=\sum_{i}^{ }\left(y_{i}-\overline{y}\right)^{2}-2\beta_{1}\sum_{i}^{ }\left(y_{i}-\overline{y}\right)\left(x_{i}-\overline{x}\right)+\beta_{1}^{2}\sum_{i}^{ }\left(x_{i}-\overline{x}\right)^{2}$$

Now we plug in $$\beta_{0}$$ and $$\beta_{1}$$ to get:

$$\sum_{i}^{ }\left(y_{i}-\overline{y}\right)^{2}-2\beta_{1}\sum_{i}^{ }\left(y_{i}-\overline{y}\right)\left(x_{i}-\overline{x}\right)+\beta_{1}^{2}\sum_{i}^{ }\left(x_{i}-\overline{x}\right)^{2}=ns_{y}^{2}-2n\frac{s_{xy}^{2}}{s_{x}^{2}}+n\frac{s_{xy}^{2}}{s_{x}^{2}}=n\left(s_{y}^{2}-2\frac{s_{xy}^{2}}{s_{x}^{2}}+\frac{s_{xy}^{2}}{s_{x}^{2}}\right)=ns_{y}^{2}\left(1-\frac{s_{xy}^{2}}{s_{x}^{2}s_{y}^{2}}\right)$$

And a little fun-fact for us here is that **$$\frac{s_{xy}}{s_{x}s_{y}}$$** is actually an important statistic called sample correlation coefficient denoted as $$\rho$$.

---
## The takeaway
$$\beta_{0}=\overline{y}-\beta_{1}\overline{x}$$

$$\beta_{1}=\frac{s_{xy}}{s_{x}^{2}}$$

$$RSS\left(\beta_{0},\beta_{1}\right)=ns_{y}^{2}\left(1-\frac{s_{xy}^{2}}{s_{x}^{2}s_{y}^{2}}\right)$$ or we can denote it now as $$RSS\left(\beta_{0},\beta_{1}\right)=ns_{y}^{2}\left(1-\rho_{xy}^{2}\right)$$

In [ ]:
beta0, beta1, rsq = fit_linear(x,y)
print("B0:", "%.2f" % beta0, "B1:", "%.2f" % beta1, "rsq:", "%.2f" % rsq)

plt.plot(x, beta1*x + beta0, color='red') # keep this in mind

plt.axis('equal')
plt.xlabel("Age (years)")
plt.ylabel("Predicted Fare ($)")
plt.show()

### What are we plotting? 

What does beta1*x + beta0 look like?
* The answer is y=mx+b, the y-int slope formula. 

We are just determining the coefficients for the feature selected and its "impact" (in quotes because we need to do much more math to determine if its actually correlated or not).

In [ ]:
# plotting the regression and the actual data on top of each other
x_pred = np.arange(-50, 100)
plt.plot(x_pred, beta1*x_pred + beta0, color='red')
plt.scatter(x,y)

plt.xlabel("Age (years)")
plt.ylabel("Fare ($)")
plt.xlim(-1, 80)
plt.grid(True)
plt.show()

Just because the data looks like it doesn't fit to a linear regression, doesn't mean that linear regression isn't useful.

You can apply transformations to the regression itself, like 1/(mx+b).